Импорт нужных либ

In [ ]:
import math

import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from prettytable import PrettyTable
%matplotlib inline

# Лабораторная работа №1. Задание 1. Вариант 25
Решается обыкновенное дифференциальное уравнение
$ U''(x) = f(x, U, U'), x \in [a; b] $
С краевыми (граничными) условиями
$ U(a) = A; $
$ \mu_1U(b) + \nu_1U'(b) = B $
Метод решения уравнения $ \varphi(\alpha) = 0 $ - метод половинного деления
Направление интегрирования: $ a \implies b $

Входные параметры

In [ ]:
# Границы отрезка, на котором ищется решение:
a, b = 1, 2
# Число отрезков разбиения
N = 50
# Точность решения нелинейного уравнения
epsilon = 1e-3
# Предельное число итераций
K = 10
# Начальное значение пристрелочного параметра
alpha = 4
# Константы краевых уравнений
A, B = 1, 8

In [ ]:
# Шаг
h = (b-a) / N
flag_IER = True
iterations_count = 0 # Для итераций в МПД
x = Symbol('x')
# u = df/dx
u = Symbol('u')
# w = du/dx
w = Symbol('w')
# Задание функции
f_exact_function = 6 * u / x**2 - 2 * w / x
f = lambdify([x, u, w], f_exact_function)
print(f_exact_function)

# Всякие функции для работы лабы

In [ ]:
# Разные функции для работы программы
def x_i():
        x_array = np.zeros(N + 1)
        for i in range(N + 1):
            x_array[i] = float(a + i * h)
        return x_array

# Функция первоначальной задачи Коши
du_dx = lambda x, u, w: w

dw_dx = f


# Метод Рунге Кутта 2 порядка:
def RungeKut(input_alpha):
    input_u = np.zeros(N + 1)
    input_w = np.zeros(N + 1)
    input_u[0] = A # u
    input_w[0] = input_alpha # w
    k1 = np.zeros(2)
    k2 = np.zeros(2)

    for i in range(N):
        k1[0] = (h * du_dx(x_array[i], input_u[i], input_w[i]))
        k1[1] = (h * dw_dx(x_array[i], input_u[i], input_w[i]))

        k2[0] = (h * du_dx(x_array[i] + h, input_u[i] + k1[0], input_w[i] + k1[1]))
        k2[1] = (h * dw_dx(x_array[i] + h, input_u[i] + k1[0], input_w[i] + k1[1]))

        input_u[i + 1] = (input_u[i] + 0.5 * (k1[0] + k2[0]))
        input_w[i + 1] = (input_w[i] + 0.5 * (k1[1] + k2[1]))
    return input_u, input_w


def fi_alpha(input_alpha):
    # В тмпу хранятся приближенные значения точного решения функции
    # В тмпв знач. первой производной функции
    tmp_u, tmp_w = RungeKut(input_alpha)
    # Значение функции на границе - В, где В - значение производной функции с права
    fi_alpha_value = tmp_w[N] - B
    return fi_alpha_value


def different_sign(lvalue, rvalue):
    return lvalue * rvalue < 0


def find_alpha2_value(alpha1):
    delta_alpha = 1.
    alpha2 = alpha1 + delta_alpha
    fi_alpha1 = fi_alpha(alpha1)
    fi_alpha2 = fi_alpha(alpha2)
    flag_reversed = False
    while not different_sign(fi_alpha1, fi_alpha2):
        alpha2 = alpha1 + delta_alpha
        fi_alpha2 = fi_alpha(alpha2)
        if fi_alpha1 < 0:
            if fi_alpha2 > fi_alpha1 and flag_reversed == False:
                delta_alpha += .2
            elif fi_alpha2 > fi_alpha1 and flag_reversed == True:
                delta_alpha -= .2
            elif fi_alpha2 < fi_alpha1:
                delta_alpha = delta_alpha * -1
                flag_reversed = True
        if fi_alpha1 > 0:
            if fi_alpha2 > fi_alpha1:
                delta_alpha = delta_alpha * -1
                flag_reversed = True
            else:
                if fi_alpha2 < fi_alpha1 and flag_reversed == False:
                    delta_alpha += .2
                if fi_alpha2 < fi_alpha1 and flag_reversed == True:
                    delta_alpha -= .2
    return alpha2

# МПД
def bisection(alpha1, alpha2, epsilon):
    alpha3 = (alpha1 + alpha2) / 2
    fi_alpha1 = fi_alpha(alpha1)
    fi_alpha2 = fi_alpha(alpha2)
    fi_alpha3 = fi_alpha(alpha3)
    iterations_count = 1
    while math.fabs(fi_alpha1 - fi_alpha2) > epsilon or iterations_count < K:
        if math.fabs(fi_alpha2) <= epsilon:
            # Возвращаем значение альфа
            return alpha2
        if different_sign(fi_alpha1, fi_alpha3):
            alpha2 = alpha3
        elif different_sign(fi_alpha3, fi_alpha2):
            alpha1 = alpha3
        fi_alpha1 = fi_alpha(alpha1)
        fi_alpha2 = fi_alpha(alpha2)
        fi_alpha3 = fi_alpha(alpha3)
        iterations_count = iterations_count + 1
    return alpha3, iterations_count

# Код алгоритма

In [ ]:
if N < 0 or K <= 0 or epsilon < 0:
    flag_IER = False
    IER = 2
    print('IER', IER, ':Неправильные значения')
while flag_IER:
    x_array = x_i()

    # Ищем значение альфа 2 для метода половинного деления
    alpha2 = find_alpha2_value(alpha)

    # Ищем значение альфа с помощью метода половинного деления
    result_alpha = bisection(alpha, alpha2, epsilon)

    # Находим итоговое решение задачи методом РК
    u_array, w_array = RungeKut(result_alpha)

    # Заголовок вывода
    if not flag_IER:
        break

    f_exact = lambda x: x**2
    f_derivative_exact = lambda x: 2*x

    th = ["x", "y", "deltaU", "y'", "deltaU'"]
    td = []
    for i in np.arange(0, N + 1):
        td.append(x_array[i])
        td.append(f_exact(x_array[i]))
        # td.append(abs(f_exact(x_array[i]) - u_array[i]))
        td.append(u_array[i])
        td.append(f_derivative_exact(x_array[i]))
        # td.append(abs(f_derivative_exact(x_array[i]) - w_array[i]))
        td.append(w_array[i])
    columns = len(th)
    table = PrettyTable(th)
    td_data = td[:]
    while td_data:
        table.add_row(td_data[:columns])
        td_data = td_data[columns:]

    print('EIR = 0:Безошибочная работа программы')
    print('Фи в альфа', fi_alpha(result))
    print('Погрешность epsilon введенная пользователем:', epsilon)
    print('Количество итераций', iterations_count)
    print('Конечное значение параметра пристрелки', result)
    print(table)

    flag_IER = False